In [1]:
# import and parameters
import torch
from torch.autograd import Variable
# load data
from tools import data_loader
import matplotlib.pyplot as plt
import numpy as np
import sys
sys.path.append('deps/sparse_rrt')
sys.path.append('.')
from sparse_rrt import _sst_module

seen_N = 1
seen_NP = 1
seen_s = 0
seen_sp = 802
data_folder = './data/cartpole_obs/'
test_data = data_loader.load_test_dataset(seen_N, seen_NP,
                      data_folder, True, seen_s, seen_sp)
obc, obs, paths, sgs, path_lengths, controls, costs = test_data
obs_width = 4.0
step_sz = 0.002
num_steps = 200
dt = 0.002
system = _sst_module.PSOPTCartPole()
cpp_propagator = _sst_module.SystemPropagator()
propagate_dynamics = lambda x, u, t: cpp_propagator.propagate(system, x, u, t)
bound = [30.0, 40.0, 3.141592653589793, 2.0]
def wrap_angle(x, system):
    circular = system.is_circular_topology()
    res = np.array(x)
    for i in range(len(x)):
        if circular[i]:
            # use our previously saved version
            res[i] = x[i] - np.floor(x[i] / (2*np.pi))*(2*np.pi)
            if res[i] > np.pi:
                res[i] = res[i] - 2*np.pi
    return res


In [2]:
# define dynamics

def dynamics(state, control):
    '''
    implement the function x_dot = f(x,u)
    return the derivative w.r.t. x
    '''
    I = 10
    L = 2.5
    M = 10
    m = 5
    g = 9.8
    H = 0.5  # cart
    # define the name for each state index and action index
    STATE_X, STATE_V, STATE_THETA, STATE_W = 0, 1, 2, 3
    CONTROL_A = 0
    # define boundary
    MIN_X = -30
    MAX_X = 30
    MIN_V = -40
    MAX_V = 40
    MIN_W = -2
    MAX_W = 2
    # obstacle information
    OBS_W = 4
    _v = state[STATE_V]
    _w = state[STATE_W]
    _theta = state[STATE_THETA]
    _a = control[CONTROL_A]
    mass_term = (M + m)*(I + m * L * L) - \
            m * m * L * L * torch.cos(_theta) * torch.cos(_theta)

    state_x = _v
    state_theta = _w
    mass_term = (1.0 / mass_term)
    # normalize: added (1/max_X) term
    #deriv[STATE_V] = (1/MAX_X)*((I + m * L * L)* \
    state_v = ((I + m * L * L)* \
        (_a + m * L * _w * _w * torch.sin(_theta)) + \
        m * m * L * L * torch.cos(_theta) * torch.sin(_theta) * g) * mass_term
    state_w = ((-m * L * torch.cos(_theta)) * \
        (_a + m * L * _w * _w * torch.sin(_theta))+(M + m) * \
        (-m * g * L * torch.sin(_theta))) * mass_term
    deriv = torch.stack([state_x, state_v, state_theta, state_w])
    return deriv


    
def enforce_state_bounds(state):
    '''

    check if state satisfies the bound
    apply threshold to velocity and angle
    return a new state toward which the bound has been enforced
    '''
    I = 10
    L = 2.5
    M = 10
    m = 5
    g = 9.8
    H = 0.5  # cart
    # define the name for each state index and action index
    STATE_X, STATE_V, STATE_THETA, STATE_W = 0, 1, 2, 3
    CONTROL_A = 0
    # define boundary
    MIN_X = -30
    MAX_X = 30
    MIN_V = -40
    MAX_V = 40
    MIN_W = -2
    MAX_W = 2
    # obstacle information
    OBS_W = 4
    """
    if state[STATE_V] < MIN_V/30.:
        new_state[STATE_V] = MIN_V/30.
    elif state[STATE_V] > MAX_V/30.:
        new_state[STATE_V] = MAX_V/30.
    """
    if(state[STATE_X]<MIN_X):
        state[STATE_X]=MIN_X
    elif(state[STATE_X]>MAX_X):
        state[STATE_X]=MAX_X

    if state[STATE_V] < MIN_V:
        state[STATE_V] = MIN_V
    elif state[STATE_V] > MAX_V:
        state[STATE_V] = MAX_V

    if state[STATE_THETA] < -np.pi:
        state[STATE_THETA] += 2*np.pi
    elif state[STATE_THETA] > np.pi:
        state[STATE_THETA] -= 2*np.pi

    if state[STATE_W] < MIN_W:
        state[STATE_W] = MIN_W
    elif state[STATE_W] > MAX_W:
        state[STATE_W] = MAX_W
    return state



def enforce_control_bounds(control):
    MIN_TORQUE, MAX_TORQUE = -300., 300.
    if control[0] < MIN_TORQUE:
        control[0] = MIN_TORQUE
    if control[0] > MAX_TORQUE:
        control[0] = MAX_TORQUE
    return control

    


def IsInCollision(x, obc, obc_width=4.):
    I = 10
    L = 2.5
    M = 10
    m = 5
    g = 9.8
    H = 0.5

    STATE_X = 0
    STATE_V = 1
    STATE_THETA = 2
    STATE_W = 3
    CONTROL_A = 0

    MIN_X = -30
    MAX_X = 30
    MIN_V = -40
    MAX_V = 40
    MIN_W = -2
    MAX_W = 2

    
    if x[0] < MIN_X or x[0] > MAX_X:
        return True
    
    H = 0.5
    pole_x1 = x[0]
    pole_y1 = H
    pole_x2 = x[0] + L * np.sin(x[2])
    pole_y2 = H + L * np.cos(x[2])

    
    for i in range(len(obc)):
        for j in range(0, 8, 2):
            x1 = obc[i][j]
            y1 = obc[i][j+1]
            x2 = obc[i][(j+2) % 8]
            y2 = obc[i][(j+3) % 8]
            if line_line_cc(pole_x1, pole_y1, pole_x2, pole_y2, x1, y1, x2, y2):
                return True
    return False

def line_line_cc(x1, y1, x2, y2, x3, y3, x4, y4):
    uA = ((x4-x3)*(y1-y3) - (y4-y3)*(x1-x3)) / ((y4-y3)*(x2-x1) - (x4-x3)*(y2-y1))
    uB = ((x2-x1)*(y1-y3) - (y2-y1)*(x1-x3)) / ((y4-y3)*(x2-x1) - (x4-x3)*(y2-y1))
    if uA >= 0. and uA <= 1. and uB >= 0. and uB <= 1.:
        # intersection
        return True
    # collision free
    return False

In [3]:
def dynamic_loss(x0, u0, x1, num_dt, dt):
    bound = torch.DoubleTensor([30.0, 40.0, 3.141592653589793, 2.0])
    x_target = x0
    for i in range(num_dt):
        x_target = dynamics(x_target, u0) * dt + x_target
    
    #x_target_norm = x_target / bound
    #x1_norm = x1 / bound  # normalize everything to -1 ~ 1
    #return torch.mean((x_target_norm - x1_norm) ** 2) / 4
    res = x_target - x1
    circular = system.is_circular_topology()

    for i in range(len(x_target)):
        if circular[i]:
            if res[i] > np.pi:
                res[i] = res[i] - 2*np.pi
            if res[i] < -np.pi:
                res[i] = res[i] + 2*np.pi
    #res = (x_target - x1) ** 2
    #res = res ** 2

    res = torch.abs(res)
    #cond = res < 1
    #l = torch.where(cond, 0.5 * res ** 2, res)
    #l = torch.mean(l, dim=0)  # sum alone the batch dimension, now the dimension is the same as input dimension
    res = res / bound
    #res = res ** 2
    res = torch.mean(res)
    #if res <= 1.:
    #    res = res * (1 / res.detach().data.item())
    return res

def bvp_solve_naive(x_init, u_init, dt, num_opt_steps, num_intermediate_steps):
    # there are num_steps - 1 intermediate nodes
    x0 = torch.from_numpy(x_init[0])
    xG = torch.from_numpy(x_init[-1])
    xs = [torch.from_numpy(x_init[i]) for i in range(len(x_init))]
    us = [torch.from_numpy(u_init[i]) for i in range(len(u_init))]
    x0 = Variable(x0, requires_grad=False)
    xG = Variable(xG, requires_grad=False)
    
    xs = [Variable(xs[i], requires_grad=False) for i in range(len(xs))]
    us = [Variable(us[i], requires_grad=True) for i in range(len(us))]
    
    #us = [Variable(us[i], requires_grad=True) for i in range(len(us))]
    #opt_algo = torch.optim.Adagrad
    opt_algo = torch.optim.LBFGS
    
    opt = opt_algo(us, lr=1e1, max_iter=2, history_size=20, tolerance_change=1e-9)#, lr=1e-1)
    
    
    max_iter = 40
    for opt_iter in range(max_iter):        
        # forward pass
        #print('iteration ', opt_iter)
        #for i in range(len(us)-2, -1, -1):
        def closure():
            for i in range(len(us)-1):
                for j in range(num_intermediate_steps):
                    xs[i+1] = dt*dynamics(xs[i], us[i]) + xs[i]
            loss = dynamic_loss(xs[-2], us[-1], xs[-1], 1, dt)
            print('loss: ', loss.item())

            opt.zero_grad()
            if loss.item() > 1e-3:
                loss.backward()
            us[1].retain_grad()
            print(us[1].grad)
            return loss
        opt.step(closure)
        with torch.no_grad():
            for i in range(len(us)):
                enforce_state_bounds(xs[i])
                enforce_control_bounds(us[i])

    x0_res = np.array([x0.data.numpy()])
    xs_res = xs[1:-1]
    xs_res = np.array([xs[i].detach().data.numpy() for i in range(len(xs_res))])
    xT_res = np.array([xG.data.numpy()])
    us_res = np.array([us[i].detach().data.numpy() for i in range(len(us))]) 
    
    #print(x0_res.shape)
    #print(xs_res.shape)
    #print(xT_res.shape)
    x_res = np.concatenate([x0_res, xs_res, xT_res], 0)
    u_res = us_res
    return x_res, u_res
        
    
    

In [4]:
import torch.nn as nn
from tools.utility import *
from model import mlp_cartpole
from model.AE import CAE_cartpole_voxel_2d
from model.mpnet import KMPNet
import os
from plan_utility import cart_pole_obs

# load MPNet
model_dir = '/media/arclabdl1/HD1/YLmiao/results/KMPnet_res/cartpole_obs_4_lr0.010000_Adagrad_step_200/'
mlp = mlp_cartpole.MLP3
cae = CAE_cartpole_voxel_2d
loss_f = nn.MSELoss()

start_epoch = 9950
direction = 0
num_steps = 200
total_input_size = 8
AE_input_size = 32
mlp_input_size = 40
output_size = 4
mpnet = KMPNet(total_input_size, AE_input_size, mlp_input_size, output_size,
               cae, mlp, loss_f)
model_path='kmpnet_epoch_%d_direction_%d_step_%d.pkl' %(start_epoch, direction, num_steps)
load_net_state(mpnet, os.path.join(model_dir, model_path))
normalize = cart_pole_obs.normalize
unnormalize = cart_pole_obs.unnormalize

[32, 32]
torch.Size([1, 7, 7, 7])
length of the output of one encoder
343


In [ ]:
import time
for envi in range(len(paths)):
    fig = plt.figure()
    ax = fig.add_subplot(111)
    #ax.set_autoscale_on(True)
    ax.set_xlim(-30, 30)
    ax.set_ylim(-np.pi, np.pi)
    new_obs_i = []
    obs_i = obs[envi]
    for k in range(len(obs_i)):
        obs_pt = []
        obs_pt.append(obs_i[k][0]-obs_width/2)
        obs_pt.append(obs_i[k][1]-obs_width/2)
        obs_pt.append(obs_i[k][0]-obs_width/2)
        obs_pt.append(obs_i[k][1]+obs_width/2)
        obs_pt.append(obs_i[k][0]+obs_width/2)
        obs_pt.append(obs_i[k][1]+obs_width/2)
        obs_pt.append(obs_i[k][0]+obs_width/2)
        obs_pt.append(obs_i[k][1]-obs_width/2)
        new_obs_i.append(obs_pt)
    obs_i = new_obs_i

    dx = 1
    dtheta = 0.1
    feasible_points = []
    infeasible_points = []
    imin = 0
    imax = int(2*30./dx)
    jmin = 0
    jmax = int(2*np.pi/dtheta)

    for i in range(imin, imax):
        for j in range(jmin, jmax):
            x = np.array([dx*i-30, 0., dtheta*j-np.pi, 0.])
            if IsInCollision(x, obs_i):
                infeasible_points.append(x)
            else:
                feasible_points.append(x)    

    feasible_points = np.array(feasible_points)
    infeasible_points = np.array(infeasible_points)
    print('feasible points')
    print(feasible_points)
    print('infeasible points')
    print(infeasible_points)
    ax.scatter(feasible_points[:,0], feasible_points[:,2], c='yellow')
    ax.scatter(infeasible_points[:,0], infeasible_points[:,2], c='pink')
    #for i in range(len(data)):
    #    update_line(hl, ax, data[i])


    
    
    #print(obs_i)
    for pathi in range(len(paths[envi])):
        start_state = sgs[envi][pathi][0]
        goal_inform_state = paths[envi][pathi][-1]
        goal_state = sgs[envi][pathi][1]
        #p = Process(target=plan_one_path, args=(obs[i], obc[i], start_state, goal_state, 500, queue))

        # propagate data
        p_start = paths[envi][pathi][0]
        detail_paths = [p_start]
        detail_controls = []
        detail_costs = []
        state = [p_start]
        control = []
        cost = []
        for k in range(len(controls[envi][pathi])):
            #state_i.append(len(detail_paths)-1)
            max_steps = int(costs[envi][pathi][k]/step_sz)
            accum_cost = 0.
            #print('p_start:')
            #print(p_start)
            #print('data:')
            #print(paths[i][j][k])
            # modify it because of small difference between data and actual propagation
            p_start = paths[envi][pathi][k]
            state[-1] = paths[envi][pathi][k]
            for step in range(1,max_steps+1):
                p_start = propagate_dynamics(p_start, controls[envi][pathi][k], step_sz)
                p_start = enforce_state_bounds(p_start)
                detail_paths.append(p_start)
                detail_controls.append(controls[envi][pathi])
                detail_costs.append(step_sz)
                accum_cost += step_sz
                if (step % 1 == 0) or (step == max_steps):
                    state.append(p_start)
                    #print('control')
                    #print(controls[i][j])
                    control.append(controls[envi][pathi][k])
                    cost.append(accum_cost)
                    accum_cost = 0.
        #print('p_start:')
        #print(p_start)
        #print('data:')
        #print(paths[i][j][-1])
        #state[-1] = paths[i][j][-1]
        data = state
        
        data = np.array(data)
        ax.scatter(data[:,0], data[:,2], c='green', s=10)
        ax.scatter(data[-1,0], data[-1,2], c='red', s=10, marker='*')
        
        start_state = data[0]
        goal_state = sgs[envi][pathi][-1]
        
        tree_nodes = [start_state.copy()]
                
        max_iter = 10
        bound_np = np.array(bound)
        sample_state = tree_nodes[0]
        num_opt_steps = 20
        num_intermediate_steps = 10
        for plan_i in range(max_iter): 
            print('iteration: ', plan_i)
            print("tree_nodes: ")
            print(tree_nodes)
            # randomly sample state in the world space
            #random_sample = np.random.uniform(low=-bound_np, high=bound_np)
            
            # find nearest neighbor in the tree
            #nearest = np.array(tree_nodes) - random_sample
            
            #print('difference:')
            #print(nearest)
            #nearest = np.linalg.norm(nearest, axis=1)
            #print("norm:")
            #print(nearest)
            #nearest = tree_nodes[np.argmin(nearest)].copy()
            #print('start point:')
            #print(nearest)
            # use MPNet to provide samples
            #nearest = tree_nodes[-1]
            bi = np.concatenate([sample_state, goal_state]).astype(np.float32)
            bi = np.array([bi])
            bi = torch.FloatTensor(bi)
            bi = normalize(bi, bound)
            bi=to_var(bi).cpu()
            bobs = np.array([obc[envi]]).astype(np.float32)
            bobs = torch.FloatTensor(bobs)
            bobs = to_var(bobs).cpu()
            #print('mpnet output: ')
            #print(mpnet(bi, bobs))
            bout = mpnet(bi, bobs)
            bout = unnormalize(bout, bound)
            
            next_state = bout.detach().data.numpy()[0]
            
            # connect from current state to next_state
            
            # initialize before solving
            x_init = np.linspace(sample_state, next_state, num_opt_steps+1)
            
            std = 0.1
            cov = np.diag([std*bound[0], std*bound[1], std*bound[2], std*bound[3]])
            #mean = next_state
            #next_state = np.random.multivariate_normal(mean=mean,cov=cov)
            mean = np.zeros(next_state.shape)
            rand_x_init = np.random.multivariate_normal(mean=mean, cov=cov, size=num_opt_steps+1)
            
            x_init = rand_x_init + x_init
            x_init[0] = sample_state
            x_init[-1] = next_state
            u_init = np.random.uniform(-300., 300., size=(num_opt_steps, 1))
            ax.scatter(x_init[:,0], x_init[:,2], c='lightgreen', s=10)
            ax.scatter(x_init[-1,0], x_init[-1,2], c='red', marker='*')
            
            
            #xs, us = bvp_solve(x_init, u_init, step_sz, num_steps)
            start_time = time.time()
            xs, us = bvp_solve_naive(x_init, u_init, step_sz, num_opt_steps, num_intermediate_steps)
            print('bvp_solver time: ', time.time() - start_time)
            #plan_one_path(obs_i, obs[i], obc[i], start_state, goal_state, goal_inform_state, 1000, data)
            #print('optimized xs:')
            #print(xs)
            #print('optimzied us:')
            #print(us)

            
            

            # propagate bvp result
            p_start = xs[0]
            detail_paths = [p_start]
            detail_controls = []
            detail_costs = []
            state = [p_start]
            control = []
            for k in range(len(us)):
                for t in range(num_intermediate_steps):
                    #state_i.append(len(detail_paths)-1)
                    # modify it because of small difference between data and actual propagation
                    p_start = propagate_dynamics(p_start, us[k], step_sz)
                    p_start = enforce_state_bounds(p_start)
                    if IsInCollision(p_start, obs_i):
                        break
                    detail_paths.append(p_start)
                    detail_controls.append(us[k])
                    state.append(p_start)
                    #print('control')
                    #print(controls[i][j])
            state = np.array(state)
            ax.scatter(state[:,0], state[:,2], c='blue', s=10)
            if len(state) > 1:
                # add results to tree
                tree_nodes.append(state[-1])
                sample_state = tree_nodes[-1]
            else:
                sample_state = tree_nodes[0]
        plt.plot()

/home/arclabdl1/anaconda3/envs/ylmiao_kino/lib/python3.6/site-packages/ipykernel/__main__.py:157: RuntimeWarning: divide by zero encountered in double_scalars
/home/arclabdl1/anaconda3/envs/ylmiao_kino/lib/python3.6/site-packages/ipykernel/__main__.py:158: RuntimeWarning: divide by zero encountered in double_scalars


feasible points
[[-30.           0.          -3.14159265   0.        ]
 [-30.           0.          -3.04159265   0.        ]
 [-30.           0.          -2.94159265   0.        ]
 ...
 [ 29.           0.           2.75840735   0.        ]
 [ 29.           0.           2.85840735   0.        ]
 [ 29.           0.           2.95840735   0.        ]]
infeasible points
[[-15.           0.           2.15840735   0.        ]
 [-15.           0.           2.25840735   0.        ]
 [-15.           0.           2.35840735   0.        ]
 ...
 [ 20.           0.          -0.74159265   0.        ]
 [ 20.           0.          -0.64159265   0.        ]
 [ 20.           0.          -0.54159265   0.        ]]
iteration:  0
tree_nodes: 
[array([-15.17990608,   0.        ,   2.66895758,   0.        ])]
loss:  0.27937712978373147
tensor([6.8492e-06], dtype=torch.float64)
loss:  0.27937712065005105
tensor([6.8492e-06], dtype=torch.float64)
loss:  0.27937712065005105
tensor([6.8492e-06], dtype=torch.flo